In [1]:
import pandas as pd
import csv
import os
import pymongo
import json
%load_ext blackcellmagic
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc, inspect, cast, Float,Integer

In [2]:
engine = create_engine("sqlite:///data/covid19.sqlite")

In [3]:
inspector = inspect(engine)
table_names = inspector.get_table_names()

# schema = inspector.get_columns("icecreamstore")
# print(schema)
table_names

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [28]:
Base = automap_base()
Base.prepare(engine, reflect=True)
global_data = Base.classes.global_covid_data
us_data = Base.classes.us_covid_data
us_lookup = Base.classes.us_lookup
session = Session(engine)

In [ ]:
schema = inspector.get_columns("global_covid_data")
# schema

In [ ]:
countries = session.query(global_data.Country_Region).distinct().order_by(global_data.Country_Region).all()

In [ ]:
global_covid_results = (
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered),
    )
    .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Country_Region)
).all()

In [ ]:
def get_country_covid_data(country):
    data = {}
    for rec in global_covid_results:
        if rec[0] == country:
            data[rec[1]] = {"Confirmed_Cases" : int(rec[2]), "Deaths" : int(rec[3]), "Recovered" : int(rec[4])}
    return data

In [ ]:
global_dict  = [{  "Country": country[0],
     "Data" : get_country_covid_data(country[0])
  }for country in countries]
global_dict

In [30]:
country = "US"
data_point = "Deaths"


In [31]:

country_results = (
    session.query(
        global_data.ID,
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    ).filter(global_data.Country_Region == country)
     .group_by(global_data.ID, global_data.Country_Region, global_data.Date)
    .order_by(global_data.Country_Region)
).all()

In [ ]:
results = [{"Date": rec[1], data_point: int(rec[col_idx])} for rec in country_results]

In [90]:
qry =(
    session.query(
        global_data.ID,
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    ).filter(global_data.Country_Region == country)
     .group_by(global_data.ID, global_data.Country_Region, global_data.Date)
    .order_by(global_data.ID)
).statement

In [92]:
df = pd.read_sql_query(qry, engine)
df.dtypes

ID                  int64
Country_Region     object
Date               object
sum_1             float64
sum_2             float64
sum_3             float64
dtype: object

In [ ]:
df_col = ""
if data_point == "Deaths":
    dc_col = df.sum_2.values.tolist()
elif data_point == "Recovered":
    df_col = df.sum_3.values.tolist()
else:
    df_col = df.sum_1.values.tolist()
data = {
    "Date": df.Date.values.tolist(),
    "Confirmed_Cases": df.sum_1.values.tolist(),
    "Deaths": df.sum_2.values.tolist(),
    "Recovered": df.sum_3.values.tolist(),
}

In [ ]:
data

## NICK S Plots

In [67]:
results = (
    session.query(
#         global_data.ID,
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    )
     .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Date)
).statement

In [85]:
df_cc = pd.read_sql_query(results, engine).rename(
    columns={
        "Country_Region": "Country",
        "sum_1": "Confirmed",
        "sum_2": "Deaths",
        "sum_3": "Recovered",
    }
).astype({'Confirmed': 'int32'})
df_cc["ID"] = df_cc.index
df_cc = df_cc.sort_values(by = "ID")
# df_cc.loc[df_cc['Country'] == "China"]
df_cc.head(30)

,Country,Date,Confirmed,Deaths,Recovered,ID
0,Afghanistan,1/22/20,0,0.0,0.0,0
1,Albania,1/22/20,0,0.0,0.0,1
2,Algeria,1/22/20,0,0.0,0.0,2
3,Andorra,1/22/20,0,0.0,0.0,3
4,Angola,1/22/20,0,0.0,0.0,4
5,Antigua and Barbuda,1/22/20,0,0.0,0.0,5
6,Argentina,1/22/20,0,0.0,0.0,6
7,Armenia,1/22/20,0,0.0,0.0,7
8,Australia,1/22/20,0,0.0,0.0,8
9,Austria,1/22/20,0,0.0,0.0,9


In [100]:
df_countrydate = df_cc[df_cc['Confirmed']>0]
df_countrydate = df_countrydate.groupby(['Date','Country']).sum().reset_index()
df_countrydate = df_countrydate.sort_values(by = "ID")
df_countrydate.dtypes


df_t = pd.read_csv("tourism1.csv")
df_t = df_t.drop(columns=['Unnamed: 0','Unnamed: 2' , 'Unnamed: 3', 'Unnamed: 4' , 'Unnamed: 5' , 'World Tourism Organization (2020), Compendium of Tourism Statistics dataset [Electronic], UNWTO, Madrid, data updated on 20/01/2020.'])
df_t = df_t.drop([0, 1, 2, 3, 4, 5])
df_t_clean = df_t.rename(columns={"Unnamed: 1": "Country", "Unnamed: 6": "Tourism"})
df_t_clean

df_merged = pd.merge(left=df_countries, right=df_t_clean, left_on='Country', right_on='Country')

In [103]:
df_countries = df_cc.groupby(['Country', 'Date']).max().reset_index().sort_values('Date', ascending=False)
df_countries = df_countries.drop_duplicates(subset = ['Country'])
df_countries = df_countries[df_countries['Confirmed']>0]

df_countries
df_t = pd.read_csv("tourism1.csv")
df_t = df_t.drop(columns=['Unnamed: 0','Unnamed: 2' , 'Unnamed: 3', 'Unnamed: 4' , 'Unnamed: 5' , 'World Tourism Organization (2020), Compendium of Tourism Statistics dataset [Electronic], UNWTO, Madrid, data updated on 20/01/2020.'])
df_t = df_t.drop([0, 1, 2, 3, 4, 5])
df_t_clean = df_t.rename(columns={"Unnamed: 1": "Country", "Unnamed: 6": "Tourism"})
df_t_clean

df_merged = pd.merge(left=df_countries, right=df_t_clean, left_on='Country', right_on='Country')

## Virus Spread

In [84]:
fig3 = px.choropleth(df_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country", 
                    animation_frame="Date"
                   )

fig3.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = .6,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig3.show()

## Total Snapshot

In [93]:

line_data = df_cc.groupby('Date').sum().reset_index()

line_data = line_data.melt(id_vars='Date', 
                 value_vars=['Confirmed', 
                             'Recovered', 
                             'Deaths'], 
                 var_name='Ratio', 
                 value_name='Value')

fig_line = px.line(line_data, x="Date", y="Value", color='Ratio', 
              title='Confirmed cases, Recovered cases, and Death Over Time')
fig_line.show()

## Tourism vs Cases

In [104]:
fig = px.scatter(df_merged, x="Tourism", y="Confirmed" , labels={'Tourism':'Total amount of people visiting country in 2019', 'Confirmed':'Total amount of COVID19 Cases'},)
fig.show()